In [3]:
import pandas as pd
from collections import defaultdict, deque

# === Load and filter lookup ===
lookup_path = 'LSOA_(2011)_to_LSOA_(2021)_to_Local_Authority_District_(2022)_Exact_Fit_Lookup_for_EW_(V3).csv'
df = pd.read_csv(lookup_path)

# df = df[df['LSOA11CD'].notna() & df['LSOA21CD'].notna()]

# === Build mappings and undirected graph ===
lsoa11_to_21 = defaultdict(set)
lsoa21_from_11 = defaultdict(set)
graph = defaultdict(set)

for _, row in df.iterrows():
    lsoa11 = row['LSOA11CD']
    lsoa21 = row['LSOA21CD']
    lsoa11_to_21[lsoa11].add(lsoa21)
    lsoa21_from_11[lsoa21].add(lsoa11)
    graph[lsoa11].add(lsoa21)
    graph[lsoa21].add(lsoa11)

# === Find connected components ===
visited = set()
components = []

for node in graph:
    if node not in visited:
        q = deque([node])
        comp = set()
        while q:
            cur = q.popleft()
            if cur in visited:
                continue
            visited.add(cur)
            comp.add(cur)
            q.extend(graph[cur] - visited)
        components.append(comp)

# === Classify each component ===
output_lines = []

for comp in components:
    lsoa11s = sorted([x for x in comp if x in lsoa11_to_21])
    lsoa21s = sorted([x for x in comp if x in lsoa21_from_11])

    if len(lsoa11s) == 1 and len(lsoa21s) == 1:
        continue  # skip 1-to-1

    if len(lsoa11s) == 1:
        if all(len(lsoa21_from_11[l21]) == 1 for l21 in lsoa21s):
            label = f"1 -> {len(lsoa21s)}"
        else:
            label = f"1 -> {len(lsoa21s)}+"
        line = f"{label}: {lsoa11s[0]} -> {', '.join(lsoa21s)}"

    elif len(lsoa21s) == 1:
        label = f"{len(lsoa11s)} -> 1"
        line = f"{label}: {', '.join(lsoa11s)} -> {lsoa21s[0]}"

    else:
        label = f"{len(lsoa11s)} <-> {len(lsoa21s)}"
        line = f"{label}: {', '.join(lsoa11s)} <-> {', '.join(lsoa21s)}"

    output_lines.append(line)

# === Save to file ===
with open("lsoa_transformation_topology.txt", "w") as f:
    f.write("\n".join(sorted(output_lines)))

In [ ]:
# 2 <-> 2: E01008187, E01027506 <-> E01035624, E01035637
# 2 <-> 2: E01023508, E01023768 <-> E01035582, E01035609
# 2 <-> 2: E01023679, E01023964 <-> E01035581, E01035608

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.affinity import translate

# Paths to shapefiles
path_2011 = "../../boundaries/Lower layer Super Output Areas (December 2011) Boundaries EW BFC (V3)/Lower_layer_Super_Output_Areas_Dec_2011_Boundaries_Full_Clipped_BFC_EW_V3_2022_1117503576712596763/LSOA_2011_EW_BFC_V3.shp"
path_2021 = "../../boundaries/Lower layer Super Output Areas (December 2021) Boundaries EW BFC (V10)/Lower_layer_Super_Output_Areas_December_2021_Boundaries_EW_BFC_V10_8562115581115271145/LSOA_2021_EW_BFC_V10.shp"

# Load shapefiles
gdf_2011 = gpd.read_file(path_2011)
gdf_2021 = gpd.read_file(path_2021)

# Define transformation pairs
complex_transformations = [
    (["E01008187", "E01027506"], ["E01035624", "E01035637"]),
    (["E01023508", "E01023768"], ["E01035582", "E01035609"]),
    (["E01023679", "E01023964"], ["E01035581", "E01035608"]),
]

# Unique colors for each polygon in a figure
colors = ['red', 'blue', 'green', 'orange']


In [10]:

# Iterate through each transformation
for i, (lsoa11_list, lsoa21_list) in enumerate(complex_transformations, 1):
    fig, ax = plt.subplots(figsize=(10, 10))

    all_codes = lsoa11_list + lsoa21_list
    all_colors = colors[:len(all_codes)]

    # Plot each 2011 LSOA with offset and color
    for idx, code in enumerate(lsoa11_list):
        poly = gdf_2011[gdf_2011['LSOA11CD'] == code]
        if not poly.empty:
            shifted = poly.translate(xoff=-100, yoff=-100)
            shifted.plot(ax=ax, edgecolor=all_colors[idx], facecolor='none', linestyle='--', linewidth=2, alpha=0.7)
            x, y = shifted.geometry.centroid.iloc[0].x, shifted.geometry.centroid.iloc[0].y
            ax.text(x, y, code, color=all_colors[idx], fontsize=8, ha='center', va='center')

    # Plot each 2021 LSOA with offset and color
    for idx, code in enumerate(lsoa21_list):
        poly = gdf_2021[gdf_2021['LSOA21CD'] == code]
        if not poly.empty:
            shifted = poly.translate(xoff=100, yoff=100)
            shifted.plot(ax=ax, edgecolor=all_colors[len(lsoa11_list) + idx], facecolor='none', linestyle='-', linewidth=2, alpha=0.7)
            x, y = shifted.geometry.centroid.iloc[0].x, shifted.geometry.centroid.iloc[0].y
            ax.text(x, y, code, color=all_colors[len(lsoa11_list) + idx], fontsize=8, ha='center', va='center')

    ax.set_title(f"Transformation {i}: {', '.join(lsoa11_list)} <-> {', '.join(lsoa21_list)}", fontsize=12)
    ax.axis('off')
    plt.tight_layout()
    plt.savefig(f"transformation_{i}_map.png", dpi=300)
    plt.close()